<a href="https://colab.research.google.com/github/hariharan-jayakumar/superres/blob/master/Super_resolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [12]:
cd /content/drive/My Drive/Projects/superres

/content/drive/My Drive/Projects/superres


In [4]:
!pip install -U -r requirements.txt

     |████████████████████████████████| 2.1MB 6.3MB/s 
Requirement already up-to-date: tensorflow>=1.13.1 in /usr/local/lib/python3.6/dist-packages (from -r requirements.txt (line 2)) (1.14.0)
     |████████████████████████████████| 1.3MB 42.3MB/s 
Requirement already up-to-date: numpy>=1.16.0 in /usr/local/lib/python3.6/dist-packages (from -r requirements.txt (line 4)) (1.16.4)
     |████████████████████████████████| 450kB 44.4MB/s 
     |████████████████████████████████| 81kB 24.8MB/s 
     |████████████████████████████████| 102kB 22.5MB/s 
     |████████████████████████████████| 92kB 27.8MB/s 
     |████████████████████████████████| 235kB 40.8MB/s 
     |████████████████████████████████| 71kB 25.8MB/s 
     |████████████████████████████████| 256kB 33.4MB/s 
     |████████████████████████████████| 204kB 45.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Stored in directory: /root/.cache/pip/wheels/61/1d/d0/04cfe49561

In [1]:
!wandb login

You can find your API keys in your browser here: https://app.wandb.ai/authorize
Paste an API key from your profile: 25a0dd02ea9af841eb8edc8364901cbf5e142596
Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


drive/  sample_data/


In [1]:
import random #to randomize the image order in random generator function
import glob #to read images from file
import subprocess #to run the LINUX command
import os #to check if path exists
from PIL import Image
import numpy as np #using numpy arrays
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import Callback
import wandb #to link the code with wandb
from wandb.keras import WandbCallback

run = wandb.init(project='superres') #connects to your project
config = run.config #the configurations for the run

config.num_epochs = 50 #number of times the model will cycle through the data
config.batch_size = 32
#size of input image
config.input_height = 32
config.input_width = 32
#size of output image
config.output_height = 256
config.output_width = 256

#address for validation and train datasets
val_dir = 'data/test'
train_dir = 'data/train'

# automatically get the data if it doesn't exist
if not os.path.exists("data"):
    print("Downloading flower dataset...")
    subprocess.check_output(
        "mkdir data && curl https://storage.googleapis.com/wandb/flower-enhance.tar.gz | tar xzf - -C data", shell=True)

#used to limit the iterations during an epoch
config.steps_per_epoch = len(
    glob.glob(train_dir + "/*-in.jpg")) // config.batch_size
config.val_steps_per_epoch = len(
    glob.glob(val_dir + "/*-in.jpg")) // config.batch_size

#it is used to return the training images following call in line 114
def image_generator(batch_size, img_dir):
    """A generator that returns small images and large images.  DO NOT ALTER the validation set"""
    #The function takes in the batch processing size and processes batch_size images at a time
    input_filenames = glob.glob(img_dir + "/*-in.jpg")
    counter = 0
    random.shuffle(input_filenames)
    #input files contain the list of all files
    while True:
        small_images = np.zeros(
            (batch_size, config.input_width, config.input_height, 3))
        large_images = np.zeros(
            (batch_size, config.output_width, config.output_height, 3))
        #allocate space for batch_size of small and large images
        if counter+batch_size >= len(input_filenames):
            counter = 0
        for i in range(batch_size): #iterates through the images
            img = input_filenames[counter + i]
            small_images[i] = np.array(Image.open(img)) / 255.0
            large_images[i] = np.array(
                Image.open(img.replace("-in.jpg", "-out.jpg"))) / 255.0
        yield (small_images, large_images)
        counter += batch_size
        #this keeps producing images to the fit_generator function in batches of batch_size


def perceptual_distance(y_true, y_pred):
    """Calculate perceptual distance, DO NOT ALTER"""
    y_true *= 255
    y_pred *= 255
    rmean = (y_true[:, :, :, 0] + y_pred[:, :, :, 0]) / 2
    r = y_true[:, :, :, 0] - y_pred[:, :, :, 0]
    g = y_true[:, :, :, 1] - y_pred[:, :, :, 1]
    b = y_true[:, :, :, 2] - y_pred[:, :, :, 2]

    return K.mean(K.sqrt((((512+rmean)*r*r)/256) + 4*g*g + (((767-rmean)*b*b)/256)))


val_generator = image_generator(config.batch_size, val_dir)
in_sample_images, out_sample_images = next(val_generator)


class ImageLogger(Callback):
    def on_epoch_end(self, epoch, logs):
        preds = self.model.predict(in_sample_images)
        in_resized = []
        for arr in in_sample_images:
            # Simple upsampling
            in_resized.append(arr.repeat(8, axis=0).repeat(8, axis=1))
        wandb.log({
            "examples": [wandb.Image(np.concatenate([in_resized[i] * 255, o * 255, out_sample_images[i] * 255], axis=1)) for i, o in enumerate(preds)]
        }, commit=False)

#we are defining a sequential model
model = Sequential()
#first layer contains 3 nodes with filter size (3,3) and activation, padding and input shape are defined
model.add(layers.Conv2D(128, (9, 9), activation='relu', padding='same',
                        input_shape=(config.input_width, config.input_height, 3)))
#we will get an output of size (config.input_width x config.input_height x 3)
model.add(layers.UpSampling2D())
#repeats the image and increases each dimension size by 2 => size is ((config.input_width x 2) x (config.input_height x 2) x 3)
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.UpSampling2D())
#size is ((config.input_width x 4) x (config.input_height x 4) x 3)
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(layers.UpSampling2D())
model.add(layers.Conv2D(3, (3, 3), activation='relu', padding='same'))
#size is ((config.input_width x 8) x (config.input_height x 8) x 3)

# DONT ALTER metrics=[perceptual_distance]
model.compile(optimizer='adam', loss='mse',
              metrics=[perceptual_distance])
#we are defining the adam optimizer to control learning rate, loss as mse and perceptual_distance as a metric


#fit_generator is an advanced version of fit
#image data can be augmented on the fly using functions
#epoch is the number of times we go through the training set
model.fit_generator(image_generator(config.batch_size, train_dir),
                    steps_per_epoch=config.steps_per_epoch,
                    epochs=config.num_epochs, callbacks=[
                        ImageLogger(), WandbCallback()],
                    validation_steps=config.val_steps_per_epoch,
                    validation_data=val_generator)

W0718 04:55:54.172057 140530776483712 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Epoch 1/50
156/156 [==============================] - 39s 253ms/step - loss: 0.0310 - perceptual_distance: 100.3288 - val_loss: 0.0170 - val_perceptual_distance: 72.3687
Epoch 2/50
156/156 [==============================] - 35s 223ms/step - loss: 0.0158 - perceptual_distance: 68.2288 - val_loss: 0.0154 - val_perceptual_distance: 67.1004
Epoch 3/50
156/156 [==============================] - 35s 222ms/step - loss: 0.0153 - perceptual_distance: 66.7092 - val_loss: 0.0150 - val_perceptual_distance: 65.4102
Epoch 4/50
156/156 [==============================] - 35s 222ms/step - loss: 0.0148 - perceptual_distance: 64.6960 - val_loss: 0.0150 - val_perceptual_distance: 65.4947
Epoch 5/50
156/156 [==============================] - 35s 222ms/step - loss: 0.0147 - perceptual_distance: 64.3046 - val_loss: 0.0149 - val_perceptual_distance: 65.0634
Epoch 6/50
156/156 [==============================] - 35s 222ms/step - loss: 0.0148 - perceptual_distance: 64.7407 - val_loss: 0.0149 - val_perceptual_dis

**First Run - 84.219**


---

Ran their code and obtained a Validation Perceptual Distance of **84.219**

**Inference:** The training loss and the validation loss are still decreasing. Hence the network can be made more complicated or it can be run for more number of epochs.

**Second Run - 62.356**

---

Modified the architecture of the network and obtained a Validation Perceptual Distance of **62.356**

**Inference:**  
Loss, perceptual distance, val loss and val perceptual distance are still decreasing very slowly. 

**Third Run - 65.7166**

---

Tried to bypass the benchmark and train on the validation images. But the network din't perform as well as it should have. It din't overtrain. So I increase the complexity of the network to make it overfit the data.  Still it is displaying similar performance
